In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.metrics import accuracy_score,confusion_matrix, log_loss

In [22]:
data_dir = "../../../kaggle_data/creditcard_overdue/"
Raw_data_dir = os.path.join(data_dir, "open/")


train=pd.read_csv(os.path.join(Raw_data_dir, "train.csv"), index_col=0)
test=pd.read_csv(os.path.join(Raw_data_dir, "test.csv"))
submit=pd.read_csv(os.path.join(Raw_data_dir, "sample_submission.csv"))

y=train['credit']
train=train.drop(axis=1, labels=['credit'])

test=test.drop(axis=1, labels=['index'])

# index 와 FLAG_MOBIL 제거
train = train.drop('FLAG_MOBIL',axis=1)
test=test.drop(axis=1,labels=['FLAG_MOBIL'])

In [23]:
train['Age'] = abs(train['DAYS_BIRTH'])/365
train = train.astype({'Age': 'int'})
for i in range(0,len(train['Age'])):
  if 20<= train['Age'][i]<30:
    train['Age'][i] = 20
  elif 30<= train['Age'][i]<40:
    train['Age'][i] = 30
  elif 40<= train['Age'][i]<50:
    train['Age'][i] = 40
  elif 50<= train['Age'][i]<60:
    train['Age'][i] = 50
  elif 60<= train['Age'][i]<70:
    train['Age'][i] = 60
train['Age'].value_counts()

<ipython-input-23-9e4ac29a532c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][i] = 30
<ipython-input-23-9e4ac29a532c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][i] = 50
<ipython-input-23-9e4ac29a532c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Age'][i] = 40
<ipython-input-23-9e4ac29a532c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

30    7585
40    6769
50    5739
20    3608
60    2756
Name: Age, dtype: int64

In [24]:
train=train.drop(axis=1, labels=['DAYS_BIRTH'])

In [25]:
test['Age'] = abs(test['DAYS_BIRTH'])/365
test= test.astype({'Age': 'int'})
for i in range(0,len(test['Age'])):
  if 20<= test['Age'][i]< 30:
    test['Age'][i] = 20
  elif 30<= test['Age'][i]< 40:
    test['Age'][i] = 30
  elif 40<= test['Age'][i]< 50:
    test['Age'][i] = 40
  elif 50<= test['Age'][i]< 60:
    test['Age'][i] = 50
  elif 60<= test['Age'][i]< 70:
    test['Age'][i] = 60
test['Age'].value_counts()

test=test.drop(axis=1, labels=['DAYS_BIRTH'])

<ipython-input-25-1323649e5906>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'][i] = 60
<ipython-input-25-1323649e5906>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'][i] = 50
<ipython-input-25-1323649e5906>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Age'][i] = 40
<ipython-input-25-1323649e5906>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

In [26]:
#문자로 된것들 라벨인코딩

train['gender'] = train['gender'].replace(['F','M'],[0,1])
test['gender'] = test['gender'].replace(['F','M'],[0,1])
print('gender :')
print(train['gender'].value_counts())
print('--------------')

print('Having a car or not : ')
train['car'] = train['car'].replace(['N','Y'],[0,1])
test['car'] = test['car'].replace(['N','Y'],[0,1])
print(train['car'].value_counts())
print('--------------')

print('Having house reality or not: ')
train['reality'] = train['reality'].replace(['N','Y'],[0,1])
test['reality'] = test['reality'].replace(['N','Y'],[0,1])
print(train['reality'].value_counts())
print('--------------')

print('edu_type')
train['edu_type'] = train['edu_type'].replace([ 'Academic degree','Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary'],[4,3,2,1,0])
test['edu_type'] = test['edu_type'].replace([ 'Academic degree','Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary'],[4,3,2,1,0])
print('--------------')

gender :
0    17697
1     8760
Name: gender, dtype: int64
--------------
Having a car or not : 
0    16410
1    10047
Name: car, dtype: int64
--------------
Having house reality or not: 
1    17830
0     8627
Name: reality, dtype: int64
--------------
edu_type
--------------


In [27]:
# income total 전처리
train['income_total'] = train['income_total'].astype(object)
train['income_total'] = train['income_total']/10000 
test['income_total'] = test['income_total']/10000

In [28]:
#나눌 개수 : n_income
n_income=10
count, bin_dividers =np.histogram(train['income_total'], bins=n_income)
bin_names=['소득'+str(i) for i in range(n_income) ]
#bin_dividers는 train기준!!
train['income_total']=pd.cut(x=train['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)
test['income_total']=pd.cut(x=test['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)

In [29]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
train['income_type']=label_encoder.fit_transform(train['income_type'])
test['income_type']=label_encoder.transform(test['income_type'])
########################################################################
train['family_type']=label_encoder.fit_transform(train['family_type'])
test['family_type']=label_encoder.transform(test['family_type'])
########################################################################
train['house_type']=label_encoder.fit_transform(train['house_type'])
test['house_type']=label_encoder.transform(test['house_type'])
########################################################################
train['income_total']=label_encoder.fit_transform(train['income_total'])
test['income_total']=label_encoder.fit_transform(test['income_total'])

In [30]:
train['occyp_type'] = train['occyp_type'].fillna("Unknown")
test['occyp_type'] = test['occyp_type'].fillna("Unknown")

train = pd.get_dummies(train, columns=['occyp_type'])
test = pd.get_dummies(test, columns=['occyp_type'])

In [31]:
#minus 변경하고
#구간화 함수
def make_bin(variable, n):
    train[variable]=-train[variable]
    test[variable]=-test[variable]
    count, bin_dividers =np.histogram(train[variable], bins=n) #train의 구간화를 적용
    bin_names=[str(i) for i in range(n)]
    train[variable]=pd.cut(x=train[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    test[variable]=pd.cut(x=test[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    test[variable].fillna(str(0), inplace=True) #test에는 없는 것을 임의의 값으로 채움
    ##########################################################
    train[variable]=label_encoder.fit_transform(train[variable])
    test[variable]=label_encoder.transform(test[variable])

In [32]:
make_bin('DAYS_EMPLOYED', n=500)
make_bin('begin_month', n=4)

In [33]:
for i in range(len(train['child_num'])):
    if train['child_num'][i]>5:
            train['child_num'][i]=6

<ipython-input-33-ccb8b74a045f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['child_num'][i]=6


In [34]:
for i in range(len(train['family_size'])):
    if train['family_size'][i]>5:
            train['family_size'][i]=6

<ipython-input-34-89d6edd72cdb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['family_size'][i]=6


In [35]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
mm=MinMaxScaler()
sc=StandardScaler()

In [37]:
train['income_total']=sc.fit_transform(train.loc[:,train.columns=='income_total'])
test['income_total']=sc.transform(test.loc[:,test.columns=='income_total'])

train['income_type']=mm.fit_transform(train.loc[:,train.columns=='income_type'])
test['income_type']=mm.transform(test.loc[:,test.columns=='income_type'])

train['edu_type']=sc.fit_transform(train.loc[:,train.columns=='edu_type'])
test['edu_type']=sc.transform(test.loc[:,test.columns=='edu_type'])

train['family_type']=mm.fit_transform(train.loc[:,train.columns=='family_type'])
test['family_type']=mm.transform(test.loc[:,test.columns=='family_type'])

train['house_type']=mm.fit_transform(train.loc[:,train.columns=='house_type'])
test['house_type']=mm.transform(test.loc[:,test.columns=='house_type'])

train['DAYS_EMPLOYED']=mm.fit_transform(train.loc[:,train.columns=='DAYS_EMPLOYED'])
test['DAYS_EMPLOYED']=mm.transform(test.loc[:,test.columns=='DAYS_EMPLOYED'])

train['family_size']=sc.fit_transform(train.loc[:,train.columns=='family_size'])
test['family_size']=sc.transform(test.loc[:,test.columns=='family_size'])

train['begin_month']=sc.fit_transform(train.loc[:,train.columns=='begin_month'])
test['begin_month']=sc.transform(test.loc[:,test.columns=='begin_month'])

train['Age']=sc.fit_transform(train.loc[:,train.columns=='Age'])
test['Age']=sc.transform(test.loc[:,test.columns=='Age'])



In [38]:
process_dir = "../result_file/preprocess_results/"

train.to_csv(os.path.join(process_dir, "preprocess_ver2_train.csv"), mode='w')
test.to_csv(os.path.join(process_dir, "preprocess_ver2_test.csv"), mode='w')